In [1]:
import numpy as np
import pandas as pd

X_train = np.load("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/erp-bci/feature extraction LinearandNonLinear/X_train.npy")
X_test = np.load("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/erp-bci/feature extraction LinearandNonLinear/X_test.npy")
Y_train = np.load("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/erp-bci/feature extraction LinearandNonLinear/Y_train.npy")
Y_test = np.load("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/erp-bci/epoched data/Y_test.npy")

In [2]:
X_train.shape,Y_train.shape, X_test.shape, Y_test.shape

((59701, 112), (59701,), (10096, 112), (10096,))

In [3]:
X_train[Y_train=='pos'].shape, X_train[Y_train=='neg'].shape

((19900, 112), (39801, 112))

In [4]:
!pip install catboost

     |████████████████████████████████| 69.2MB 60kB/s 


In [5]:
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
!pip install optuna

     |████████████████████████████████| 296kB 2.1MB/s 
     |████████████████████████████████| 174kB 14.2MB/s 
     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 112kB 14.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 143kB 14.4MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=717bd35fa81502b0332be5b7c099d4982c1ae9eebb3b25d3394a73cb1d2b2732
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [50]:
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score

In [75]:
def objective(trial):
    #X_train, X_test, y_train, y_test = train_test_split(train,test, test_size=0.3)

    param = {
        "iterations" : trial.suggest_int("iterations", 10,1000),
        "depth": trial.suggest_int("depth", 4,16),
        "learning_rate" : trial.suggest_float("learning_float", 0.01, 1),
        "class_weights":class_weights
    }
    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(X_test)
    # pred_labels = np.rint(preds)
    # accuracy = accuracy_score(Y_test, preds)
    score = recall_score(Y_test,preds,pos_label='pos')
    # return accuracy
    return score

In [76]:
X_train.shape,Y_train.shape,X_test.shape, Y_test.shape

((59701, 112), (59701,), (10096, 112), (10096,))

In [1]:
study = optuna.create_study(direction="maximize") #create_study creates a bigger space where traiing occurs and importanve is give to maximize accuracy
study.optimize(objective,timeout=600)

NameError: ignored

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial
print

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [55]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(Y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=Y_train)
class_weights = dict(zip(classes, weights))
class_weights

{'neg': 0.7499937187507851, 'pos': 1.5000251256281407}

In [9]:
from catboost import CatBoostClassifier

In [62]:
model=CatBoostClassifier(task_type='CPU', depth= 4, iterations= 35, learning_rate=0.09011926395992339,class_weights=class_weights)
model.fit(X_train, Y_train)

0:	learn: 0.6922260	total: 60.1ms	remaining: 2.04s
1:	learn: 0.6916531	total: 105ms	remaining: 1.73s
2:	learn: 0.6909846	total: 152ms	remaining: 1.62s
3:	learn: 0.6902650	total: 197ms	remaining: 1.53s
4:	learn: 0.6896709	total: 244ms	remaining: 1.46s
5:	learn: 0.6891884	total: 290ms	remaining: 1.4s
6:	learn: 0.6885240	total: 337ms	remaining: 1.35s
7:	learn: 0.6878064	total: 381ms	remaining: 1.29s
8:	learn: 0.6874205	total: 427ms	remaining: 1.23s
9:	learn: 0.6870570	total: 469ms	remaining: 1.17s
10:	learn: 0.6863128	total: 517ms	remaining: 1.13s
11:	learn: 0.6860203	total: 570ms	remaining: 1.09s
12:	learn: 0.6856974	total: 618ms	remaining: 1.04s
13:	learn: 0.6851513	total: 662ms	remaining: 994ms
14:	learn: 0.6844987	total: 705ms	remaining: 940ms
15:	learn: 0.6841154	total: 752ms	remaining: 893ms
16:	learn: 0.6837440	total: 802ms	remaining: 849ms
17:	learn: 0.6832268	total: 842ms	remaining: 795ms
18:	learn: 0.6829124	total: 885ms	remaining: 745ms
19:	learn: 0.6825354	total: 928ms	remaini

In [63]:
from sklearn.metrics import classification_report,confusion_matrix
y_pred=model.predict(X_test)
from sklearn.metrics import confusion_matrix,balanced_accuracy_score
cr = classification_report(Y_test,  y_pred )
print(cr)

              precision    recall  f1-score   support

         neg       0.86      0.98      0.91      8661
         pos       0.13      0.02      0.04      1435

    accuracy                           0.84     10096
   macro avg       0.49      0.50      0.47     10096
weighted avg       0.75      0.84      0.79     10096



In [61]:
confusion_matrix(Y_test,y_pred)

array([[6733, 1928],
       [1103,  332]])

0.5043758978833789

In [ ]:
y_pred2 = model.predict(X_train)
cr = classification_report(Y_train,  y_pred2 )
print(cr)

              precision    recall  f1-score   support

         neg       0.68      0.99      0.81     39801
         pos       0.85      0.07      0.13     19900

    accuracy                           0.69     59701
   macro avg       0.76      0.53      0.47     59701
weighted avg       0.74      0.69      0.58     59701



In [ ]:
confusion_matrix(Y_train,y_pred2)

array([[39552,   249],
       [18524,  1376]])

In [ ]:
optuna.visualization.plot_param_importances(study)
